# VGG16 Transfer Learning on MNIST Digit Recognition Dataset

## Import libraries

In [107]:
import numpy as np 
import pandas as pd 
import os, time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import os

## Config GPU

In [122]:
# Config using GPU 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15593763496092381809
]


In [123]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


AttributeError: module 'keras.backend' has no attribute 'tensorflow_backend'

## Load Dataset & Preprocessing
Preprocessing: 
* Convert image from `1` channels to `3` channels
* Resize image from `(28,28)` to `(48,48)`
* Normalize dataset
* For label, using one-hot encoding


In [108]:
# read csv file
data_train = pd.read_csv(r"./data/train_new.csv") 
data_test = pd.read_csv(r"./data/test_new.csv") 
data_train.head()
data_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
# shape
print('Train: ',data_train.shape)
print('Test: ',data_test.shape) 

Train:  (29400, 785)
Test:  (12600, 784)


In [110]:
# select & convert to numpy 
X_train = np.array(data_train.iloc[:,1:])
y_train = np.array(data_train.iloc[:,0])
X_test = np.array(data_test.iloc[:,0:])
y_test = np.array(data_test.iloc[:,0])

In [111]:
# convert from df to np
# X_train = data_train.iloc[:,1:].to_numpy()
# X_test = data_test.iloc[:,1:].to_numpy()
# print(X_train.shape,X_test.shape)

In [112]:
# convert vector img to 28x28
X_train = X_train.reshape(X_train.shape[0], 28,28)
X_test = X_test.reshape(X_test.shape[0], 28,28)
print(X_train.shape)
print(X_test.shape)

(29400, 28, 28)
(12600, 28, 28)


In [113]:
# one-hot encoding 
print('Number of classes: ',len(np.unique(y_train)) )
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)
print(Y_train_oh)

Number of classes:  10
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [114]:
# Convert from 1 channel to 3 channel
# X_train = np.dstack([X_train] * 3)
# X_test = np.dstack([X_test] * 3)
X_train = np.stack((X_train,)*3, axis = -1)
X_test = np.stack((X_test,)*3, axis = -1)

print('Train ',X_train.shape)
print('Test ',X_test.shape)

Train  (29400, 28, 28, 3)
Test  (12600, 28, 28, 3)


In [115]:
# # Reshape to tensor 
X_train = X_train.reshape(-1,28,28,3)
X_test = X_test.reshape(-1,28,28,3)

# resize 28x28 to 48x48 required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_test])

print(X_train.shape, X_test.shape)

(29400, 48, 48, 3) (12600, 48, 48, 3)


In [116]:
# Normalize dataset
X_train = X_train / 255.
X_test = X_test / 255.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [117]:
# train test split 
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train_oh,                                                            test_size= 0.3, 
                                                    random_state=42)

In [118]:
X_train.shape, y_train.shape

((20580, 48, 48, 3), (20580, 10))

# Load VGG16 pretrained model & modified model

## Feature Extraction

In [66]:
# attach FC1 (28,28,3) -> eval 
# attach FC1 -> FC2 -> eval
model = VGG16(weights='imagenet', input_shape=(48,48,3), include_top=False)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [121]:
train_features = model.predict(np.array(X_train), batch_size=8, verbose=1)
test_features = model.predict(np.array(X_test), batch_size=8, verbose=1)
val_features = model.predict(np.array(X_valid), batch_size=8, verbose=1)

 728/2573 [=======>......................] - ETA: 2:12

KeyboardInterrupt: 

## Training

## Training with Image Augmentation

## Evaluation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

##  Generate submission

In [ ]:
result = model.predict(df_test.to_numpy())
image_id = []
for i in range(df_test.shape[0]):
  image_id.append(i + 1)
image_id = np.array(image_id)
result_df = pd.DataFrame({'ImageId': image_id, 'Label': result})
result_df.to_csv('result.csv')